# Hashtag Grouping on Twitter

Input: 
    Hastag scrap result from cekmedsos_database.vw_ttidata

Output: 
    Mapping tables to group each hashtag entry with similarity
    

What we need to do....
1. Load the table entry from mySQL into python
2. Read the entry

Pre-requisite
pip install wheel
pip install pandas

### 1. Create Connection to mySQL

ref to this page:
    [How to Use Python with mySQL in Jupyter](https://medium.com/@tattwei46/how-to-use-python-with-mysql-79304bee8753)
    
first, we need to install mysql connector

In [1]:
import pymysql.cursors
import pandas as pd
import uuid
import time
import os

# db parameters
_conn_host='202.157.185.40'
_conn_user='cekmedsos_db'
_conn_password='282E~f0si'
_conn_database='cekmedsos_database'

### Input as proces started
Record header and parameter information

 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal
 *
 
 * Kesepakatan jenis analisa AI
 * 1 --> Analisa Cluster
 * 2 --> Analisa image clustering
 * 3 --> Analisa sentiment
 
 

In [2]:
from datetime import datetime

# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# get available jobs from database server, first come first serve
sql = "select id, hastag, `parameter` \
from screen_analisis_ai \
where active = 1 \
and status = 1 \
and jenis_analisa = 1 \
order by created asc, id asc limit 1"

row_count = cursor.execute(sql)

if(row_count == 0):
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()

result = cursor.fetchall()
database_keyword_id = result[0]['hastag']
similarity_treshold = result[0]['parameter']
i_process_id = result[0]['id']
screen_name = ''

print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)

87276
0.7
2922


### Marking this process as running
so that another process will not take this one


In [3]:
# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"

# execute
cursor.execute(sql, i_process_id)

# commit record
connection.commit()

2922


### Runnning Process


In [4]:
# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (%s, %s, %s)"
# Execute the query
print(i_process_id)
cursor.execute(sql, (str(i_process_id), datetime.now(), 1 ))

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
cursor.execute(sql, (i_process_id, 1, 'Similarity Treshold', similarity_treshold))
cursor.execute(sql, (i_process_id, 1, 'DB_ID', database_keyword_id))

# Commit Record
connection.commit()

2922


#### Starting process, 
Run query against RDBMS


In [5]:
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

def execute_mysql_query(query):
    try:
        # Define the connection parameters inside the function
        host = "202.157.185.40"
        port = 3306  # Replace with your MySQL port number
        database = "cekmedsos_database"
        user = "cekmedsos_db"
        password = "282E~f0si"

        # Create a SQLAlchemy engine using the provided connection parameters
        connection_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(connection_url)

        # Establish a connection
        connection = engine.connect()

        # Execute the MySQL query and fetch the results into a DataFrame
        result_df = pd.read_sql(query, connection)

        # Close the database connection
        connection.close()

        print("Query executed successfully.")

        return result_df

    except SQLAlchemyError as e:
        print(f"Error: {str(e)}")
        return None

In [6]:
#
# Query to get tweet data, apply analitics to this dataset
#

# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '
# s_query_string = 'select id, tweet, tweet_date_time from vw_ret_tweet_clean where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + 'db_id = "' + database_keyword_id.replace('"','') + '"'
    
print(s_query_string)
df = execute_mysql_query(s_query_string)

# Close Connection
connection.close()

# see result
df.head()

select id, tweet, tweet_date_time from ret_tweet where db_id = "87276"
Query executed successfully.


,id,tweet,tweet_date_time
0,1311577856237993984,"Terjun ke politik, apakah Gibran Rakabuming ak...",2020-10-01 15:05:03
1,1649315880277508100,PDI Perjuangan Berjuang Untuk Kesejahteraan Ra...,2023-04-21 14:35:23
2,1698550530136772970,Tidak Perlu Anti dalam Politik\n\nSaya termasu...,2023-09-04 11:16:18
3,1699412712005193992,rekomendasi kaos yang pernah dipake nct ????\n...,2023-09-06 20:22:18
4,1699427771724837189,rekomendasi ide menu bekal simple ✨\n\n— a thr...,2023-09-06 21:22:09


### 2. Try to pre-process all the text

target-> tokenizing into another dataframe

In [7]:
import string
import re
import nltk

from nltk.tokenize import word_tokenize

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    
    return strOut
# end Text Preprocessing


# Apply to data frame
df['tweet'] = df['tweet'].apply(text_preproc)

df.head()

,id,tweet,tweet_date_time
0,1311577856237993984,terjun ke politik apakah gibran rakabuming aka...,2020-10-01 15:05:03
1,1649315880277508100,pdi perjuangan berjuang untuk kesejahteraan ra...,2023-04-21 14:35:23
2,1698550530136772970,tidak perlu anti dalam politik saya termasuk y...,2023-09-04 11:16:18
3,1699412712005193992,rekomendasi kaos yang pernah dipake nct — a th...,2023-09-06 20:22:18
4,1699427771724837189,rekomendasi ide menu bekal simple ✨ — a thread...,2023-09-06 21:22:09


### Finish preprocessing, Tokenized
Next step, is to output to new column for tokenized sentence

In [8]:
from nltk.corpus import stopwords 

# Load stopwords
sw = stopwords.words('indonesian')

# apply tokenize
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

# apply stopword removal
df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

df.head()

,id,tweet,tweet_date_time,tokenized_tweet,sw
0,1311577856237993984,terjun ke politik apakah gibran rakabuming aka...,2020-10-01 15:05:03,"[terjun, ke, politik, apakah, gibran, rakabumi...","{matanajwaberebuttakhtaditengahwabah, klik, ht..."
1,1649315880277508100,pdi perjuangan berjuang untuk kesejahteraan ra...,2023-04-21 14:35:23,"[pdi, perjuangan, berjuang, untuk, kesejahtera...","{ber, kesejahteraan, mengalami, sumpah, hatimu..."
2,1698550530136772970,tidak perlu anti dalam politik saya termasuk y...,2023-09-04 11:16:18,"[tidak, perlu, anti, dalam, politik, saya, ter...","{anti, nasdem, parpol, anies, bukti, membawa, ..."
3,1699412712005193992,rekomendasi kaos yang pernah dipake nct — a th...,2023-09-06 20:22:18,"[rekomendasi, kaos, yang, pernah, dipake, nct,...","{nct, httpstcoqhgcmcpfak, dipake, —, a, kaos, ..."
4,1699427771724837189,rekomendasi ide menu bekal simple ✨ — a thread...,2023-09-06 21:22:09,"[rekomendasi, ide, menu, bekal, simple, ✨, —, ...","{ide, lemonindonesia, bekal, cr, ig, simple, —..."


### Define function to calculate similarity
Function return similarity score

In [9]:
def calculate_similarity(X_set, Y_set):
# Program to measure the similarity between  
# two sentences using cosine similarity. 

    l1 =[];l2 =[]

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i]
    try:
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
    except:
        # print('zero div on X_set')
        return 0;
        
    return cosine

def largest_in_col(arr,nCol):
    # 
    # Find largest value of col nCol on 2D arr
    #
    
    # init value
    max_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] > max_val:
            max_val = arr[x][nCol]
            row_index = x
        
    return max_val,row_index

def smallest_in_col(arr,nCol):
    # 
    # Find smallest value of col nCol on 2D arr
    #
    
    # init value
    min_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] < min_val:
            min_val = arr[x][nCol]
            row_index = x
        
    return min_val,row_index


### Try to using function
using some of array cells, create n matrices

1. take one tweet, compare to all data set
2. flag 1 if similar
3. take next tweet, if similar from prev tweet, skip
4. if not similar, add counter, then proceed

In [10]:
import numpy as np
import array as arr
from tqdm import tqdm

# set to max CPU Cores
# multitasking.set_max_threads(multitasking.config["CPU_CORES"] * 5)

st = similarity_treshold
cluster_no = 1
s_score = 0
s_score_current = 0
i_current_cluster = 0

#create zero element array
#col 0 => index base tweet
#col 1 => cluster number
#col 2 => similarity score
base_tweet = []

#proceed to compare to all tweet
cosine_matrix = np.zeros(( len(df), len(df) ), dtype=np.dtype('f4'))

# flag the mt
with tqdm(total=( (len(df)*len(df)/2))-(len(df)/2))  as pbar:
    for j in range(0, len(df)):
        tweet_to_compare = df['sw'][j]

        #check, is this second tweet?
        if(j == 0):
            #first tweet, add as cluster no #1
            base_tweet.append([j,1,1.0])
            i_current_cluster = base_tweet[0][1]

        elif(j == 1):
            #compare to prev tweet
            s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[0][0] ])

            if(s_score < st):
                #not similar
                base_tweet.append([j,2,1])
                i_current_cluster = base_tweet[j][1]

        else:
            #other else tweet
            for x in range(0,len(base_tweet)):
                #compare every element
                s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[x][0] ])
                base_tweet[x][2] = s_score

            if(largest_in_col(base_tweet,2)[0] < st):
                #no similar, add as one new cluster
                i_current_cluster = i_current_cluster + 1
                base_tweet.append([j,i_current_cluster,largest_in_col(base_tweet,2)[0]])
            else:
                #determine cluster# from biggest similarity
                i_current_cluster = base_tweet[(largest_in_col(base_tweet,2)[1])][1]

        #proceed to compare to all tweet
        for i in range(0,len(df)):
            # update progress
            if (j<i):
                pbar.update(1)
                s_score = calculate_similarity(tweet_to_compare, df['sw'][i])
                if (s_score >= st):
                    cosine_matrix[i,j] = i_current_cluster


pbar.close()
print(cosine_matrix)

100%|██████████| 64031586/64031586.0 [20:28<00:00, 52116.37it/s]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Writing result to file
Using NPZ format for efficiency, and try to load them after save


In [11]:
# save numpy array as npz file
from numpy import asarray
from numpy import savez_compressed

# save to npy file
# savez_compressed('./output/df_380.npz',df)
# savez_compressed('./output/data_380.npz', cosine_matrix)
# savez_compressed('./output/data_380_base.npz', base_tweet)

print(len(cosine_matrix))
print('Save data complete .... ')

11317
Save data complete .... 


### How to save to rdbms?
save the header data, meta data of the process
[ret_analysis_header]
- JobID
- user initiated
- Time Started
- Time End

Process Parameter
[ret_analysis_parameter]
- JobID
- Param Name
- Param Value
    - ST Value
    - DB ID
    - Screen Name
    - How Many Tweet analyzed

save detail cluster information
[ret_base_tweet]
- JobID
- Tweet Base# --> tweet ID
- Cluster#

save detail calculation result, structure
[ret_cluster_result]
- JobID
- TweetID
- Cluster

In [12]:
# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Base Tweet Record
sql = "insert into ret_base_tweet (job_id, tweet_id, cluster_id) values (%s, %s, %s)"

## inserting base taweet
for i in range(0,len(base_tweet)):
    # Execute the query
    cursor.execute(sql, (i_process_id, df['id'][i], base_tweet[i][1]))
    
connection.commit()
connection.close()

print('finished inserting base tweet record')

finished inserting base tweet record


### Record cluster result
into table ret_cluster_result

In [13]:
from numpy import asarray
from numpy import savetxt

savetxt('data.csv', cosine_matrix, delimiter=',')

In [14]:
# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Tweet Cluster Record
sql = "insert into ret_cluster_result (job_id, tweet_id, cluster_no) values (%s, %s, %s)"

print(len(cosine_matrix))
print(i_process_id)

# initiate cluster number
i_cluster_no_save = 0
temp_val = 0
                 
## inserting tweet cluster
for i in range(0, len(cosine_matrix)):
    
    # find value in this particular row
    for j in range(0,len(cosine_matrix[i])):
        
        # print(cosine_matrix[i][j])
        temp_val = cosine_matrix[j][i]
        
        if(temp_val != 0):
            i_cluster_no_save = temp_val

    # Execute the query
    cursor.execute(sql, (i_process_id, df['id'][i], i_cluster_no_save))
    
connection.commit()
connection.close()

print('finished inserting cluster data')

11317
2922
finished inserting cluster data


### Record finish time
update table ret_analysis_header

In [15]:
# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
cursor.execute(sql, (i_process_id, 1, '#Tweet Processed',len(df)))

#
# Create Tweet Cluster Record
sql = "update ret_analysis_header set datetime_finish = %s where job_id = %s"
# Executing query
cursor.execute(sql, (datetime.now(),i_process_id) )

print(i_process_id)

connection.commit()
connection.close()

print('job finished')

2922
job finished


### Marking as finished the job
on this particular i_process_id

In [16]:
# Connect to the database
connection = pymysql.connect(host=_conn_host,
                             user=_conn_user,
                             password=_conn_password,
                             database=_conn_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# inserting jobs into table mv result
sql = "call spInsertResultToMV(%s);" 
# Executing query
cursor.execute(sql,i_process_id)

sql = "update screen_analisis_ai set status = 3, end_process = now(), duration = TIMESTAMPDIFF(second,start_process, end_process) where id = %s"
# Executing query
cursor.execute(sql,i_process_id)

# commit changes
connection.commit()
connection.close()

print('inserting result finished')



inserting result finished


### Done Here ...
1. Create data structure to record below things
    Job Header
    - JobID
    - Similarity Treshold
    - Dataset Parameter
        - By User
        - By Keyword
    - Running time start
    - Running time end
    
    Dataset Result
        - base_tweet
        - cosine_matrix
        
2. Push the result data into RDBMS Server, in this case is mySQL
    - upload csv file to mySQL
    - import to database
  
Entry on crontab
* * * * * /home/haviz/ai-project/tweet-grouping/run_one.sh >> /home/haviz/ai-project/tweet_gorup.log 2>&1

In [17]:
# Wait 10 sec before release
time.sleep(10)